In [75]:
import os,pathlib,csv,dotenv,json,requests,time, datetime as dt
from typing import Dict, List
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv


path = r"C:\Users\Kartavya\bootcamp_Kartavya_Joshi\homework\homework\homework4"
os.chdir(path)
os.getcwd()

DATA_RAW = pathlib.Path("data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

dotenv.load_dotenv()

True

In [64]:
API_ALPHAKEY= os.getenv("API_KEY_ALPHAVANTAGE")
API_ALPHAKEY

'T6ZCSVD6PVSQGHFM'

Helper functions: File name, time stamp and validation

In [67]:
def savestamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

In [69]:
def save_filename(prefix: str, meta: Dict[str,str]) -> str:
    mid="_".join([f"{k}-{str(v).replace(' ', '-')[:20]}" for k,v in meta.items()])
    return f"{prefix}_{mid}_{save_stamp()}.csv"

def validate_df(df: pd.DataFrame, required_cols:List[str], dtypes_map: Dict[str,str]) ->Dict[str,str]:
    msgs= {}
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing cols'] =f"Missing columns: {missing}"
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                    if dtype == 'datetime64[ns]':
                        pd.to_datetime(df[col])
                    elif dtype == 'float':
                        pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    return msgs                 

API Ingestion

In [95]:
try:
    symbol = ['NVDA']
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={API_ALPHAKEY}"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    js = r.json()

    key = [k for k in js.keys() if "Time Series" in k]
    assert key, f"Unexpected response keys: {list(js.keys())}"

    series = js[key[0]]
    df_api = (
        pd.DataFrame(series).T
        .rename_axis("date")
        .reset_index()
        .rename(columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        })
    )

    print(" Data from Alpha Vantage API")
    print(df_api.head())

except Exception as e:
    print(" API failed, falling back to Yahoo Finance:", e)
    
    import yfinance as yf
    df_api = yf.download(symbol, period="6mo", interval="1d").reset_index()[['Date','Close']]
    df_api.columns = ['date','close']

    
df_api.to_csv("data/raw/NVDA_20250819", index=False)

 API failed, falling back to Yahoo Finance: Unexpected response keys: ['Error Message']
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [99]:
scrape_url = f"https://pmc.ncbi.nlm.nih.gov/articles/PMC2533031/table/T1/"
headers = {"User-Agent": "AFE-Course-Notebook/1.0"}
        
res = requests.get(scrape_url, headers=headers, timeout=20)
res.raise_for_status()
soup = BeautifulSoup(res.text, "html.parser")
table = soup.find("table")
rawdata = []

for tr in table.find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cells:
        rawdata.append(cells)
   
header, *data = rawdata[1:]
df_scrape = pd.DataFrame(data, columns=header)
    
print(" Scraping sample data: Juice concentrate nutrition info:")
print(df_scrape.head())
    
    
#save result to CSV in data/raw
df_scrape.to_csv("data/raw/scrape_sampletable", index=False)

 Scraping sample data: Juice concentrate nutrition info:
                       mg/kg   mg/d  μmol/d      %
0            Vitamin C   360  255.5  1451.9   2.3%
1          Carotenoids  8.10    5.7    11.5  18.5%
2        Xanthophylls1  5.40    3.8     6.7  18.1%
3      Cryptoxanthins2  1.59    1.1     2.0  16.2%
4  Trans beta-carotene  0.40    0.3     0.5  22.2%


Validation -text, date and number

In [101]:
msgs2 = validate_df(df_scrape, required_cols=list(df_scrape.columns), dtypes_map={})
print(msgs2)

{'na_total': 'Total NA values: 0'}


Assumptions and Risk:
Risk:
1) yfinance API getting crashed.
2) scrape url not being fetched

Assumptions:
1) 